In [1]:
import pandas as pd
import error_prompts as p
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd

# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()

prompt_name = 'error_class_ML'
prompt = p.error_class_ML

prompt_name = 'logistic_regression'

file = "logistic_regression"

DATA_PATH = f'../../Datasets/Evaluations/Schema Matching/{file}.csv'
OUTPUT_PATH = f"../../Datasets/Evaluations/Schema Matching/Error_Analysis/error_class_LLM_analysis.csv"

MODEL = "gpt-4-turbo-preview"
TEMP = 0


KeyboardInterrupt: 

In [7]:
import os
df = pd.read_csv(DATA_PATH)
print(df.shape)
print(df.head())

df['error'] = None
df['error'] = ['correct' if y_true == y_pred else 'error' for y_true, y_pred in zip(df['y_true'], df['y_pred'])]


(450, 12)
   table_index                      table_name  column_index_left  \
0            0  29021592_3_2299138476894681059                  0   
1            1  29021592_3_2299138476894681059                  3   
2            2  29021592_3_2299138476894681059                  4   
3            3  29021592_3_2299138476894681059                  2   
4            4  29021592_3_2299138476894681059                  0   

   column_index_right  y_true  y_pred  \
0                   0       1       0   
1                   0       0       0   
2                   0       0       0   
3                   0       0       0   
4                   3       0       0   

                                      column_table_A  \
0        Domitian | Nerva | Hadrian | Trajan | Titus   
1  Titus Flavius Domitianus | Marcus Cocceius Ner...   
2              81-96 | 96-8 | 117-8 | 98-117 | 79-81   
3                        nan | nan | nan | nan | nan   
4        Domitian | Nerva | Hadrian | Trajan | T

In [8]:
df_correct = df[df['error'] == 'correct']
df_correct_samples = df_correct.sample(n=5, random_state=42)

df_error = df[df['error'] == 'error']
df_error_samples = df_error.sample(n=20, random_state=42)

print(df_correct_samples.shape, df_error_samples.shape)
print(df_correct_samples.head())
print(df_error_samples.head())

if os.path.exists(OUTPUT_PATH):
    df_answer = pd.read_csv(OUTPUT_PATH)
else:
    df_answer = pd.DataFrame(columns=['Prompt_Name', 'Prompt', 'Model', 'Database', 'Answer', 'Context'])

df_correct_samples = df_correct_samples.reset_index(drop=True)
df_error_samples = df_error_samples.reset_index(drop=True)






(5, 13) (20, 13)
     table_index                      table_name  column_index_left  \
18            18  34899692_0_6530393048033763438                  3   
11            11  29021592_3_2299138476894681059                  2   
229          229  86297395_0_6919201319699354263                  4   
293          293   3917335_0_7791699395300625164                  2   
150          150  48944826_0_2321751364268052533                  4   

     column_index_right  y_true  y_pred  \
18                    1       0       0   
11                    5       0       0   
229                   4       0       0   
293                   5       0       0   
150                   0       0       0   

                                        column_table_A  \
18   Peseta (ESP) | Dollar (TWD) | Koruna (CZK) | B...   
11                         nan | nan | nan | nan | nan   
229  $10,661 | $71,904.24 | $2,584.63 | $3,027.85 |...   
293              110,910 | 458 | 603,700 | 14 | 29,800   
150    

In [9]:
def execute(df, prompt, df_answer):
    i = 1
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, timeout=150)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, timeout=150)
    examples = ""
    examples += "Wrong Decisions:\n"
    for j in range(len(df_error_samples)):
        if str(df_error_samples.iloc[j]['y_pred']) == '1':
            pred_match = "match"
        else:
            pred_match = "do not match"

        if str(df_error_samples.iloc[j]['y_true']) == '1':
            true_match = "match"
        else:
            true_match = "do not match"
        examples += f"Task {i}:\n"
        examples += "Tables: \n Table A:\n" + df_error_samples.iloc[j]['table_A'] + "\nTable B:\n" + df_error_samples.iloc[j]['table_B'] + "\n"
        examples += "Predicted answer for Specific Column Pair: Column A-" + str(df_error_samples.iloc[j]['column_index_left']) + " and Column B-" + str(df_error_samples.iloc[j]['column_index_right']) + " " + pred_match + "\n"
        examples += "Explanation: " + df_error_samples.iloc[j]['token_weights'] + "\n"
        examples += "Correct answer for Specific Column Pair: Column A-" + str(df_error_samples.iloc[j]['column_index_left']) + " Column B-" + str(df_error_samples.iloc[j]['column_index_right']) + " " + true_match  + "\n\n\n"
        i += 1
    examples += "\nCorrect Decisions (Only as a reference):\n"
    for j in range(len(df_correct_samples)):
        if str(df_correct_samples.iloc[j]['y_pred']) == 'True':
            pred_match = "match"
        else:
            pred_match = "do not match"

        if str(df_correct_samples.iloc[j]['y_true']) == 'True':
            true_match = "match"
        else:
            true_match = "do not match"
        examples += f"Task {i}:\n"
        examples += "Tables: \n Table A:\n" + df_correct_samples.iloc[j]['table_A'] + "\nTable B:\n" + df_correct_samples.iloc[j]['table_B'] + "\n"
        examples += "Predicted answer for Specific Column Pair: Column A-" + str(df_correct_samples.iloc[j]['column_index_left']) + " and Column B-" + str(df_error_samples.iloc[j]['column_index_right']) + " " + pred_match + "\n"
        examples += "Explanation: " + df_correct_samples.iloc[j]['token_weights'] + "\n"
        examples += "Correct answer for Specific Column Pair: Column A-" + str(df_error_samples.iloc[j]['column_index_left']) + " Column B-" + str(df_error_samples.iloc[j]['column_index_right']) + " " + true_match  + "\n"
        i += 1
    chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
    try: 
        result = chain.run(examples = str(examples), callbacks=[handler])
    except Exception as e:
        print(e)
        print(chain.prompt.format_prompt(examples = str(examples)).to_string())
    handler.langfuse.flush()
    prompt_text = chain.prompt.format_prompt(examples = str(examples)).to_string()
    print(prompt_text)
    print("\n")
    print(result)
    print("\n")
    new_row = {'Prompt_Name': prompt_name, 'Prompt': prompt_text, 'Model': MODEL, 'Database': file, 'Answer': result, 'Context': 'logistic_regression'}
    df_new_row = pd.DataFrame([new_row])
    df_answer = pd.concat([df_answer, df_new_row])
    return result, df_answer


answer, df_answer = execute(df, prompt, df_answer)
    



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In the following I will give you a schema matching tasks together with a column matching decision, details about the decision and the if the columns actually match.
The decision was made by a Logistic Regression Machine Learning Model and the additional information shows the weights the model assigned to the different tokens. 
Can you please group the wrong decisions into distinct fault categories? Please also indicate how often each one occurs.
There are also some correct decisions in the examples. Please just use them as a reference and don't categorize them.
Wrong Decisions:
Task 1:
Tables: 
 Table A:
|   Column A-0 |   Column A-1 |   Column A-2 | Column A-3   | Column A-4   |   Column A-5 | Column A-6   | Column A-7           |
|-------------:|-------------:|-------------:|:-------------|:-------------|-------------:|:-------------|:---------------------|
|           11 |       265.91 |        22.29 | UK           | Oil and gas  |       198.14 | 217.60       | BP                   

In [10]:
print(df_answer)

           Prompt_Name                                             Prompt  \
0      error_class_LLM  In the following I will give you schema matchi...   
1      error_class_LLM  In the following I will give you schema matchi...   
2      error_class_LLM  In the following I will give you schema matchi...   
0  logistic_regression  In the following I will give you a schema matc...   

                 Model                                           Database  \
0  gpt-4-turbo-preview  unstructured_fix_fewshot_multi_table_gpt-4-tur...   
1  gpt-4-turbo-preview  unstructured_dynamic_fewshot_multi_table_gpt-4...   
2  gpt-4-turbo-preview  structured_unstructured_zero_shot_multi_table_...   
0  gpt-4-turbo-preview                                logistic_regression   

                                              Answer                Context  
0  Based on the provided examples and decisions, ...  unstructured_analysis  
1  Fault Categories and Occurrences:\n\n1. **Mism...  unstructured_analy

In [14]:
df_answer.to_csv(OUTPUT_PATH, index=False)